In [2]:
import json
import re

In [3]:
def parse_line(line: str):
    data = json.loads(line)
    return (data["Text"], data["Hubs"])

In [4]:
rdd = sc.textFile("hdfs://master:54310/data_no_duplicates")

In [5]:
parsed = rdd.map(lambda row: parse_line(row))

In [6]:
def parse_words(text: str):
    without_non_word = re.sub("\W", " ", text)
    without_spaces = re.sub("(\s)+", " ", without_non_word)
    return without_spaces.strip().split(" ")

In [7]:
splitted_by_words = parsed.map(lambda bit: (parse_words(bit[0]), bit[1]))

In [8]:
def select_cartesians(doc):
    return [(word,topic)
               for word in doc[0]
               for topic in doc[1]]

In [9]:
pairs = splitted_by_words.flatMap(lambda tup: select_cartesians(tup))
pairs.take(1)

[('Добрый', 'Я пиарюсь')]

In [14]:
groupped = pairs.groupBy(lambda pair: pair[0])

In [26]:
from collections import Counter
def count_topics(topics):
    counts = Counter(topics)
    return [(t, counts[t]) for t in topics]

In [30]:
def map_bad_tuples(tuples):
    return [t[1] for t in tuples]

In [31]:
t = groupped. \
    map(lambda g: (g[0], map_bad_tuples(list(g[1])))). \
    map(lambda g: (g[0], count_topics(g[1])))

[('предпосылка',
  [('Биотехнологии', 1),
   ('Блог компании RoKiCom', 1),
   ('Сетевые технологии', 3),
   ('Cisco', 1),
   ('Системное администрирование', 1),
   ('Обработка изображений', 1),
   ('Блог компании Мосигра', 1),
   ('Чёрная дыра', 4),
   ('Чёрная дыра', 4),
   ('Delphi', 1),
   ('Программирование', 5),
   ('Блог компании Инфопульс Украина', 1),
   ('Робототехника', 1),
   ('Медиа', 1),
   ('Программирование', 5),
   ('Программирование', 5),
   ('Чёрная дыра', 4),
   ('Учебный процесс в IT', 1),
   ('Железо', 1),
   ('Game Development', 1),
   ('Разработка мобильных приложений', 1),
   ('Сетевые технологии', 3),
   ('Сетевые технологии', 3),
   ('OpenStreetMap', 1),
   ('JavaScript', 2),
   ('Программирование', 5),
   ('Программирование', 5),
   ('Интерфейсы', 1),
   ('Чёрная дыра', 4),
   ('Разработка', 1),
   ('jQuery', 1),
   ('JavaScript', 2),
   ('Криптография', 1),
   ('Беспроводные технологии', 3),
   ('Беспроводные технологии', 3),
   ('Беспроводные технологии', 3